In [0]:
!pip install fake_useragent

In [0]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup

In [0]:
url = "https://movie.naver.com/movie/running/current.nhn#"
response = requests.get(url)

soup = BeautifulSoup(response.content,'lxml')
soup = soup.find('div', id='content').find('div',class_='lst_wrap')
#print(soup)
category = soup.find_all('li')
img_url = [cg.img['src'] for cg in category] #
url_list = [cg.dl.a['href'] for cg in category] #
title = [cg.dl.a.text for cg in category] #
age_limit = [cg.dl.span.text for cg in category] #
star_point = [cg.dl.find('dd', class_='star').find('span',class_='num').text for cg in category] #
genre = [cg.dl.find('dl',class_='info_txt1').a.text for cg in category] #
#director = [cg.dl.find('dl',class_='info_txt1').find('span', class_='link_txt').next_sibling.
#            next_sibling.next_sibling.strip() for cg in category]
director = [cg.dl.find('dl',class_='info_txt1').find('dt', class_='tit_t2').next_sibling.next_sibling.
            a.text for cg in category] #
review_title = [] #
review_content = [] #

#actor = [cg.dl.find('dl',class_='info_txt1').find('dt', class_='tit_t3').next_sibling.next_sibling.a.text for cg in category]
#actor_access = category.dl.find('dl',class_='info_txt1').find('dt', class_='tit_t3').next_sibling.next_sibling
#actor = [aa.find_all('a').text for aa in actor_access]
#print(img_url)
#print(url_list)
#print(title)
#print(age_limit)
#print(star_point)
#print(genre)
#print(director)
idx = 1
story = []
for in_url in url_list:
  inner_url = "https://movie.naver.com/{}".format(in_url)
  response = requests.get(inner_url)
  soup = BeautifulSoup(response.content,'lxml')
  temp_story = soup.find('div',class_='story_area').p.text

  while temp_story.find('\n') > -1:
      temp_story = temp_story.replace('\n','')
  while temp_story.find('\t') > -1:
      temp_story = temp_story.replace('\t','')
  
  story.append(temp_story)

  review_url = "https://movie.naver.com/{}".format(in_url)
  review_url = review_url.replace('basic','review')
  code_text = review_url[review_url.find('code=') + 5:]

  review_response = requests.get(review_url)
  review_soup = BeautifulSoup(review_response.content,'lxml')
  has_review = review_soup.find('div',class_='top_behavior').span.text
  has_review = has_review[1:-1]
  if has_review == '0':
    review_title.append('No reviews')
    review_content.append('No reviews')
    continue
  else:
    onclick_text = review_soup.find('ul',class_='rvw_list_area').a.attrs['onclick']
    onclick_text = onclick_text[onclick_text.find(';') + 19:].replace(')','')
    #review_inner_url = review_url[]+"&order=#tab"
    review_inner_url = "https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid={}&code={}&order=#tab".format(onclick_text,code_text)
    #review_inner_url = review_inner_url.replace('review.nhn?','reviewread.nhn?nid={}&'.format(onclick_text))
    reviewread_response = requests.get(review_inner_url)
    reviewread_soup = BeautifulSoup(reviewread_response.content,'lxml')
    review_title.append(reviewread_soup.find('div',class_='obj_section noline center_obj').find('strong',class_='h_lst_tx').text)
    temp_review_content = reviewread_soup.find('div',class_='user_tx_area').text.strip()

    while temp_review_content.find('\n') > -1:
      temp_review_content = temp_review_content.replace('\n','')

    while temp_review_content.find('\t') > -1:
      temp_review_content = temp_review_content.replace('\t','')

    review_content.append(temp_review_content)

  if idx > 11:
    break
  
  idx = idx+1
    

  
#print(story)
#print(actor)

In [8]:
print(len(story))
print(len(title))
print(len(review_content))
print(len(review_title))
print(len(star_point))
print(len(director))

12
95
12
12
95
95


In [0]:
print(review_url)

https://movie.naver.com//movie/bi/mi/review.nhn?code=109607


In [0]:
tes = 'abcd'
print(tes.find('a'))
print(tes.find('e'))

0
-1


In [0]:
import csv

df=pd.DataFrame
info_list = []
# .tsv 쓰기
#f = open('movie_info.tsv', 'w', encoding='euc-kr', newline='')
#wr = csv.writer(f, delimiter='\t')
for num in range(1,len(review_content)-1):
  info_list.append([title[num], star_point[num], director[num], genre[num], review_title[num], review_content[num], story[num], age_limit[num], img_url[num]])

df = pd.DataFrame(info_list, columns = ['title','star_point', 'director', 'genre', 'review_title', 'review_content', 'story', 'age_limit', 'img_url' ])
#wr.writerow(df)

#time_pd = pd.DataFrame(0., columns=col, index=time_range)

#time_pd.to_csv("filename.csv", mode='w')

#df.to_csv
df.to_csv("movie_info.csv", mode='w', encoding='UTF-8', sep='\t')
df.to_csv("movie_info.tsv", mode='w', encoding='UTF-8', sep='\t')



In [0]:
nums = 4
print(title[nums])
print(star_point[nums])
print(director[nums])
print(genre[nums])
print(review_title[nums])
print(review_content[nums])
print(age_limit[nums])
print(img_url[nums])

스타워즈: 라이즈 오브 스카이워커
7.34
J.J. 에이브럼스
액션
e
e
12세 관람가
https://movie-phinf.pstatic.net/20200108_148/1578462346140MWVbW_JPEG/movie_image.jpg?type=m99_141_2


In [0]:
df

,title,star_point,director,genre,review_title,review_content,age_limit,img_url
0,닥터 두리틀,6.82,스티븐 개건,코미디,2019년 5월 26일 첫 리뷰,캬.. 로버트다우니주니어와 톰홀랜드의 만남이라니.. 벌써부터 기대가 된다.수능 치고...,전체 관람가,https://movie-phinf.pstatic.net/20191226_207/1...
1,나쁜 녀석들 : 포에버,7.65,아딜 엘 아르비,액션,"윌 스미스,코믹 액션 <나쁜 녀석들> 후속 시리즈에 제작만 참여?","﻿최근 소니 픽쳐스가 개봉일을 확정한 코믹 액션 <나쁜 녀석들>의 세번째,네번째 속...",청소년 관람불가,https://movie-phinf.pstatic.net/20200107_50/15...
2,백두산,5.94,이해준,드라마,백두산 좌파빨갱이들의 영화를 언제까지 봐야하나?,영화 : 백두산 리뷰한국영화의 개돼지세뇌가 어디까지 가는지 모르겠다. 자신들의 종교...,12세 관람가,https://movie-phinf.pstatic.net/20191219_263/1...
3,스타워즈: 라이즈 오브 스카이워커,7.34,J.J. 에이브럼스,액션,"스타워즈 라이즈 오브 스카이워커 후기, 아이맥스에서 본 내 결말은..",불가능한거는 없어요!- 영화 스타워즈 라이즈 오브 스카이워커 에서..\n\n\n ...,12세 관람가,https://movie-phinf.pstatic.net/20200108_148/1...
4,천문: 하늘에 묻는다,8.82,허진호,허진호,"영화 천문 후기, 실화 같은 결말이 주는 재미가 있다!",조선의 것으로 조선에 맞는 것을 만들면 됩니다- 영화 천문에서..\n\n\n ☆...,12세 관람가,https://movie-phinf.pstatic.net/20191231_207/1...
5,타오르는 여인의 초상,9.81,셀린 시아마,드라마,[제 24회 부산국제영화제] 타오르는 여인의 초상(Portrait of a Lady...,"타오르는 여인의 초상\n\n감독\n셀린 시아마\n출연\n발레리아 골리노, 아델 하에...",15세 관람가,https://movie-phinf.pstatic.net/20191220_49/15...
6,겨울왕국 2,8.70,크리스 벅,애니메이션,"겨울왕국2 해석 (Frozen2, 2019)",스토리 핵심은 00입니다.※영화를 보지 않은 분들에게는 스포일러가 될 수 있으니 주...,전체 관람가,https://movie-phinf.pstatic.net/20191121_221/1...
